You can write notes & updates here:

.
.



In [1]:
#loading libraries & data files
library(tidyverse)
library(tidymodels)
library(lubridate)

players_data <- read_csv("players.csv")
sessions_data <- read_csv("sessions.csv")


── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.1     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.1
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors
── Attaching packages ────────────────────────────────────── tidymodels 1.1.1 ──

✔ broom        1.0.6     ✔ rsample      1.2.1
✔ dials        1.3.0     ✔ tune         1.1.2
✔ infer        1.0.7     ✔ workflows    1.1.4
✔ modeldata    1.4.0     ✔ workflowsets 1.0.1
✔ parsnip      1.2.1     ✔ yardstick    1.3.1
✔ recipes      1.1.0     

── Conflicts ───────────────────────────────────────── tidymodels_conflicts() ──
✖ scales::discard() masks purrr::discard()
✖ dplyr::filt

In [4]:
#tidying data
#tidy data for players data
players_data <- mutate(players_data,experience=as.factor(experience))
players_data <- mutate(players_data,gender=as.factor(gender))

#tidy data for length of time
sessions_data_time <- mutate(sessions_data,start_time=dmy_hm(start_time))
sessions_data_time <- mutate(sessions_data_time,end_time=dmy_hm(end_time))

sessions_data_plustime <- mutate(sessions_data_time,time= end_time-start_time)
#slice(sessions_data_plustime, 1:5)

sessions_data_numeric_time <- mutate(sessions_data_plustime,diff_time= as.numeric(time))
#slice(sessions_data_numeric_time, 1:5)

#actual tidy data for use
sessions_data_tidy <- select(sessions_data_numeric_time, hashedEmail, start_time, end_time, diff_time)
head(sessions_data_tidy)



hashedEmail,start_time,end_time,diff_time
<chr>,<dttm>,<dttm>,<dbl>
bfce39c89d6549f2bb94d8064d3ce69dc3d7e72b38f431d8aa0c4bf95ccee6bf,2024-06-30 18:12:00,2024-06-30 18:24:00,12
36d9cbb4c6bc0c1a6911436d2da0d09ec625e43e6552f575d4acc9cf487c4686,2024-06-17 23:33:00,2024-06-17 23:46:00,13
f8f5477f5a2e53616ae37421b1c660b971192bd8ff77e3398304c7ae42581fdc,2024-07-25 17:34:00,2024-07-25 17:57:00,23
bfce39c89d6549f2bb94d8064d3ce69dc3d7e72b38f431d8aa0c4bf95ccee6bf,2024-07-25 03:22:00,2024-07-25 03:58:00,36
36d9cbb4c6bc0c1a6911436d2da0d09ec625e43e6552f575d4acc9cf487c4686,2024-05-25 16:01:00,2024-05-25 16:12:00,11
bfce39c89d6549f2bb94d8064d3ce69dc3d7e72b38f431d8aa0c4bf95ccee6bf,2024-06-23 15:08:00,2024-06-23 17:10:00,122


In [3]:
#counting number of sessions for each individual
# this method is not the most straightforward
#will be updated later, so just ignore it for now


session_timeone <- sessions_data_plustime <- mutate(sessions_data_numeric_time, onetime=1)
#head(session_timeone)

session_num <- pivot_wider(session_timeone, names_from=hashedEmail, values_from=onetime)
#colnames(session_num)
#tail(session_num)

session_count_horiz<- summarize(session_num,across(bfce39c89d6549f2bb94d8064d3ce69dc3d7e72b38f431d8aa0c4bf95ccee6bf:bc704ff2bc676dbf48ee41b9e11481c1387bf758ad318f2428f336e3fecc6660,~ sum(.x, na.rm = TRUE)))
session_count<- pivot_longer(session_count_horiz, cols=bfce39c89d6549f2bb94d8064d3ce69dc3d7e72b38f431d8aa0c4bf95ccee6bf:bc704ff2bc676dbf48ee41b9e11481c1387bf758ad318f2428f336e3fecc6660, names_to="hashedEmail", values_to="num_sessions")
#nrow(session_count)

session_count_over2 <- filter(session_count, num_sessions>1)
one_session <- nrow(session_count)-nrow(session_count_over2)
#print(one_session)
#print("81 out of 125 individuals have only 1 gaming session")